In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [6]:
import numpy as np

In [9]:
import pandas as pd

df = pd.read_csv("spam.csv",encoding="ISO-8859-1")


In [10]:
df.drop(["Unnamed: 2","Unnamed: 3","Unnamed: 4"],axis="columns",inplace=True)

In [11]:
df["Message"]=df["v2"]
df["spam"]=df["v1"].apply(lambda x: 1 if x=="spam" else 0)
df.head()
df.drop(["v1","v2"],axis="columns",inplace=True)

In [12]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df["Message"],df["spam"], stratify=df["spam"])

In [15]:
len(x_train)

4179

In [3]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [4]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs["pooled_output"])
l = tf.keras.layers.Dense(1, activation="sigmoid", name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [5]:
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [16]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
131/131 [==============================] - 666s 5s/step - loss: 0.3250 - accuracy: 0.8686
Epoch 2/5
131/131 [==============================] - 694s 5s/step - loss: 0.2414 - accuracy: 0.8899
Epoch 3/5
131/131 [==============================] - 697s 5s/step - loss: 0.2004 - accuracy: 0.9198
Epoch 4/5
131/131 [==============================] - 697s 5s/step - loss: 0.1766 - accuracy: 0.9308
Epoch 5/5
131/131 [==============================] - 680s 5s/step - loss: 0.1592 - accuracy: 0.9445


In [17]:
model.evaluate(x_test, y_test)

44/44 [==============================] - 219s 5s/step - loss: 0.1534 - accuracy: 0.9411


[0.15338392555713654, 0.9411342144012451]

In [18]:
model.save("sms_flagger")

INFO:tensorflow:Assets written to: sms_flagger\assets


INFO:tensorflow:Assets written to: sms_flagger\assets


In [21]:
tf.keras.models.save_model(new_model,"reduced_model")

INFO:tensorflow:Assets written to: reduced_model\assets


INFO:tensorflow:Assets written to: reduced_model\assets


In [22]:
# from tensorflow import keras
# new_model =keras.models.load_model("reduced_model")